In [45]:
from langchain.chains import ConversationalRetrievalChain
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from typing import Any, Dict

In [50]:
from types import SimpleNamespace


settings = {
    'OPENAI_API_KEY': 'YOUR_API_KEY',
}

settings = SimpleNamespace(**settings)

## Set up Guardrails Output Parser

The prompt element in the Guardrails output parser is the prompt of the final chain in the pipeline. This is the prompt that will be used to generate the Guardrails output.

In this case, this is the prompt for the QA chain.

In [3]:
BOTSONIC_TEMPLATE = """
<rail version="0.1">
    <output>
        <string name="result" description="chatbot's response" required="true"/>
    </output>

    <instructions>
        You are a helpful bot, you are provided with matching data from the knowledge base related to the query from the user. You are to help users provide information properly.
        
        @json_suffix_prompt_examples
    </instructions>

    <prompt>
    @xml_prefix_prompt

    {output_schema}
    </prompt>
</rail>
"""

output_parser = GuardrailsOutputParser.from_rail_string(BOTSONIC_TEMPLATE, num_reasks=0)

## Set up ConversationalRetrieverChain

In [4]:
# Botsonic team to set up the following:
DEFAULT_SYSTEM_PROMPT = "You are having a conversation with an AI assistant. The assistant is here to help you with your questions. The assistant is very knowledgeable about the topic you are discussing."
MODEL = "gpt-3.5-turbo"

### Instantiate sub chains

#### Question Generator Chain

In [5]:
llm = ChatOpenAI(
    model_name=MODEL, temperature=0, openai_api_key=settings.OPENAI_API_KEY
)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

#### Question Answering Chain

**Note:**

This is the final chain in the pipeline. It is responsible for generating the answer to the question.
Therefore, the prompt for this chain needs to be updated in order to include Guardrails style prompting.

In [36]:
# Setting up prompt

system_template = SystemMessagePromptTemplate.from_template("""
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}

{format_instructions}
""")

human_msg_template = HumanMessagePromptTemplate.from_template("""
Answer the question below, and return a JSON that follows the correct schema.

{question}

{format_instructions_human_msg}"
""")

CHAT_PROMPT = ChatPromptTemplate(
    messages=[system_template, human_msg_template],
    input_variables=["context", "question"],
    partial_variables={
        "format_instructions": output_parser.guard.rail.instructions.format_instructions,
        "format_instructions_human_msg": output_parser.guard.rail.prompt.format_instructions,
    },
)

print(CHAT_PROMPT.format(context="This is a test context", question="This is a test question"))

System: 
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
This is a test context


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `None`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" f

In [37]:
streaming_llm = ChatOpenAI(
    model_name=MODEL,
    streaming=True,
    verbose=True,
    temperature=0,
    openai_api_key=settings.OPENAI_API_KEY,
)
doc_chain = load_qa_chain(streaming_llm, chain_type="stuff", prompt=CHAT_PROMPT)

In [38]:
doc_chain.llm_chain.prompt.format(context="This is a test context", question="This is a test question")

'System: \nUse the following pieces of context to answer the users question. \nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n----------------\nThis is a test context\n\n\nONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML\'s tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `None`.\n\nHere are examples of simple (XML, JSON) pairs that show the expected behavior:\n- `<string name=\'foo\' format=\'two-words lower-case\' />` => `{{\'foo\': \'example one\'}}`\n- `<list name=\'bar\'><string format=\'upper-case\' /></list>` => `{{"bar": [\'STRING ONE\', \'STRING TWO\', etc.]}}`\n- `<object name=\'baz\'><string name="foo" format="capitalize t

### Instantiate Retriever

In [39]:
loader = TextLoader("docs/modules/state_of_the_union.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=settings.OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents, embeddings)

Using embedded DuckDB without persistence: data will be transient


### Instantiate ConversationalRetrieverChain

In [40]:
qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    return_source_documents=True,
)

## Run ConversationalRetrieverChain with Guardrails Output Parser

In [47]:
def run(question, chat_history) -> Dict[str, Any]:
    res = qa({"question": "What did the president say about Ketanji Brown Jackson", "chat_history": ""})
    return output_parser.parse(res["answer"])

In [48]:
res = run(
    question="What did the president say about Ketanji Brown Jackson",
    chat_history=""
)

In [49]:
print(res)

{'result': 'The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson, who is one of our nation’s top legal minds and will continue Justice Breyer’s legacy of excellence. She has received a broad range of support since she’s been nominated, from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.'}
